In [56]:

import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

# Cấu hình hiển thị số đẹp
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print("🚀 KHỞI ĐỘNG HỆ THỐNG PHÂN CỤM FINAL (ROBUST + INTERACTIVE)...")

🚀 KHỞI ĐỘNG HỆ THỐNG PHÂN CỤM FINAL (ROBUST + INTERACTIVE)...


In [57]:
# --- 1. TỰ ĐỘNG TÌM VÀ LOAD DỮ LIỆU ---
def load_data():
    # Danh sách các đường dẫn có thể chứa file
    possible_paths = {
        'trans': ["cleaned_uk_data.csv", "data/processed/cleaned_uk_data.csv"],
        'rules': ["rules_apriori_filtered.csv", "data/processed/rules_apriori_filtered.csv"]
    }
    
    path_trans = next((p for p in possible_paths['trans'] if os.path.exists(p)), None)
    path_rules = next((p for p in possible_paths['rules'] if os.path.exists(p)), None)

    if not path_trans or not path_rules:
        # Báo lỗi rõ ràng nếu thiếu file
        raise FileNotFoundError("❌ KHÔNG TÌM THẤY FILE! Hãy đảm bảo bạn đã upload 'cleaned_uk_data.csv' và 'rules_apriori_filtered.csv' vào cùng thư mục.")
        
    print(f"✅ Đã nạp dữ liệu thành công:\n   - {path_trans}\n   - {path_rules}")
    df_c = pd.read_csv(path_trans, parse_dates=["InvoiceDate"])
    df_r = pd.read_csv(path_rules)
    
    # Fix lỗi chuỗi list trong cột antecedents (do lưu csv bị biến thành string)
    df_r['antecedents'] = df_r['antecedents'].apply(lambda x: list(eval(x)) if isinstance(x, str) else x)
    return df_c, df_r

In [58]:
def run_premium_visualization(X, rfm_original, k=4):
    # Chạy K-Means
    print(f"\n🎨 Đang vẽ bộ biểu đồ Premium (K={k})...")
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X)
    
    # Chuẩn bị dữ liệu vẽ
    plot_data = rfm_original.copy()
    plot_data['Cluster'] = labels.astype(str) # Chuyển thành chuỗi để phân loại màu
    plot_data['Cluster_Label'] = "Nhóm " + plot_data['Cluster'] # Nhãn đẹp hơn
    
    # --- 1. BIỂU ĐỒ 3D SCATTER (CÓ KÝ HIỆU) ---
    print("   -> 1. Vẽ 3D Scatter với ký hiệu khác nhau...")
    pca = PCA(n_components=3)
    coords = pca.fit_transform(X)
    plot_data['PC1'], plot_data['PC2'], plot_data['PC3'] = coords[:,0], coords[:,1], coords[:,2]
    
    fig_3d = px.scatter_3d(
        plot_data, x='PC1', y='PC2', z='PC3',
        color='Cluster_Label',
        symbol='Cluster_Label',  # <--- QUAN TRỌNG: Ký hiệu khác nhau cho từng cụm
        hover_data=['Recency', 'Frequency', 'Monetary'],
        title='<b>PHÂN CỤM KHÁCH HÀNG 3D (ĐA KÝ HIỆU)</b>',
        color_discrete_sequence=px.colors.qualitative.Bold, # Bảng màu đậm, rõ nét
        opacity=0.8
    )
    fig_3d.update_traces(marker=dict(size=5, line=dict(width=1, color='DarkSlateGrey'))) # Viền cho điểm
    fig_3d.update_layout(scene=dict(xaxis_title='PCA 1', yaxis_title='PCA 2', zaxis_title='PCA 3'))
    fig_3d.show()

    # --- 2. RADAR CHART (CHÂN DUNG CHI TIẾT) ---
    print("   -> 2. Vẽ Radar Chart so sánh...")
    scaler = MinMaxScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(rfm_original), columns=rfm_original.columns)
    df_scaled['Cluster'] = labels
    means = df_scaled.groupby('Cluster').mean().reset_index()
    
    categories = ['Recency (Gần nhất)', 'Frequency (Tần suất)', 'Monetary (Chi tiêu)']
    fig_radar = go.Figure()
    
    # Bảng màu thủ công để đồng bộ
    colors = ['#FF6633', '#FFB399', '#FF33FF', '#FFFF99', '#00B3E6']
    
    for i, row in means.iterrows():
        color_val = colors[i % len(colors)]
        fig_radar.add_trace(go.Scatterpolar(
            r=[row['Recency'], row['Frequency'], row['Monetary']],
            theta=categories,
            fill='toself',
            name=f'Nhóm {int(row["Cluster"])}',
            line=dict(color=color_val, width=2),
            marker=dict(symbol='circle', size=6)
        ))
        
    fig_radar.update_layout(
        polar=dict(radialaxis=dict(visible=True, range=[0, 1], gridcolor='lightgrey')),
        title="<b>CHÂN DUNG TỪNG NHÓM (RADAR CHART)</b>",
        showlegend=True,
        template="plotly_white"
    )
    fig_radar.show()

In [59]:
# --- 3. SO SÁNH THUẬT TOÁN (YÊU CẦU NÂNG CAO - ĐIỂM 10) ---
def compare_algorithms_interactive(X, k=4):
    print(f"\n⚔️ Đang so sánh K-Means, Agglomerative và DBSCAN (K={k})...")
    
    # 1. K-Means
    km = KMeans(n_clusters=k, random_state=42, n_init=10).fit(X)
    score_km = silhouette_score(X, km.labels_)
    
    # 2. Agglomerative
    agg = AgglomerativeClustering(n_clusters=k).fit(X)
    score_agg = silhouette_score(X, agg.labels_)
    
    # 3. DBSCAN
    db = DBSCAN(eps=5.0, min_samples=5).fit(X)
    # Xử lý trường hợp DBSCAN ra toàn nhiễu (-1)
    if len(set(db.labels_)) > 1:
        score_db = silhouette_score(X, db.labels_)
    else:
        score_db = 0 
        
    # Vẽ biểu đồ so sánh
    results = pd.DataFrame({
        'Algorithm': ['K-Means', 'Agglomerative', 'DBSCAN'],
        'Score': [score_km, score_agg, score_db]
    })
    
    fig = px.bar(results, x='Algorithm', y='Score', color='Score',
                 title='<b>SO SÁNH HIỆU QUẢ THUẬT TOÁN (SILHOUETTE SCORE)</b>',
                 color_continuous_scale='Viridis', text_auto='.3f')
    fig.show()
    
    return km.labels_ # Trả về nhãn K-Means để phân tích tiếp

In [60]:
# --- 4. TRỰC QUAN HÓA TƯƠNG TÁC (VISUALIZATION) ---
def run_visualizations(X, rfm_original, labels):
    # A. 3D SCATTER PLOT
    print("🧊 Vẽ biểu đồ 3D tương tác...")
    pca = PCA(n_components=3)
    coords = pca.fit_transform(X)
    plot_df = rfm_original.copy()
    plot_df['Cluster'] = labels.astype(str)
    plot_df['PC1'], plot_df['PC2'], plot_df['PC3'] = coords[:,0], coords[:,1], coords[:,2]
    
    fig = px.scatter_3d(plot_df, x='PC1', y='PC2', z='PC3', color='Cluster',
                        hover_data=['Recency', 'Frequency', 'Monetary'],
                        title='<b>PHÂN CỤM KHÁCH HÀNG 3D (RFM + RULES)</b>', 
                        opacity=0.7, size_max=10)
    fig.show()
    
    # B. RADAR CHART (CHÂN DUNG KHÁCH HÀNG)
    print("🕸️ Vẽ Radar Chart...")
    scaler = MinMaxScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(rfm_original), columns=rfm_original.columns)
    df_scaled['Cluster'] = labels
    means = df_scaled.groupby('Cluster').mean().reset_index()
    
    fig = go.Figure()
    categories = ['Recency', 'Frequency', 'Monetary']
    for i, row in means.iterrows():
        fig.add_trace(go.Scatterpolar(
            r=[row['Recency'], row['Frequency'], row['Monetary']],
            theta=categories,
            fill='toself', name=f'Cluster {int(row["Cluster"])}'
        ))
    fig.update_layout(polar=dict(radialaxis=dict(visible=True, range=[0, 1])), 
                      title="<b>CHÂN DUNG KHÁCH HÀNG (RADAR CHART)</b>")
    fig.show()
    
    # C. SNAKE PLOT (HÀNH VI)
    print("🐍 Vẽ Snake Plot...")
    df_melt = pd.melt(means, id_vars=['Cluster'], value_vars=['Recency', 'Frequency', 'Monetary'])
    fig = px.line(df_melt, x='variable', y='value', color='Cluster', markers=True,
                  title="<b>SNAKE PLOT (HÀNH VI KHÁCH HÀNG)</b>")
    fig.show()

In [ ]:

if __name__ == "__main__":
    try:
        # 1. Load & Process Data
        df_t, df_r = load_data()
        rfm_raw, final_feat, X_input = process_data(df_t, df_r, top_k=100)
        
        if len(X_input) < 10:
            print("⚠️ CẢNH BÁO: Dữ liệu quá ít sau khi lọc! Hãy kiểm tra lại file input.")
        else:
            # 2. So sánh & Chọn thuật toán (Mặc định chọn K=4 để demo)
            # Bạn có thể đổi số k=... ở đây
            best_labels = compare_algorithms_interactive(X_input, k=4)
            
            # 3. Vẽ biểu đồ chi tiết
            run_visualizations(X_input, rfm_raw, best_labels)
            
            # 4. Báo cáo kết quả
            print("\n📋 BÁO CÁO HỒ SƠ KHÁCH HÀNG (PROFILING):")
            summary = rfm_raw.copy()
            summary['Cluster'] = best_labels
            stats = summary.groupby('Cluster').agg({
                'Recency': 'mean', 
                'Frequency': 'mean', 
                'Monetary': 'mean', 
                'Cluster': 'count'
            }).rename(columns={'Cluster': 'Số lượng KH'}).round(1)
            display(stats)
            
            print("\n✅ HOÀN TẤT BẢN FINAL! Đã xuất toàn bộ biểu đồ tương tác.")
            
    except Exception as e:
        print(f"\n❌ LỖI XẢY RA: {e}")

✅ Đã nạp dữ liệu thành công:
   - cleaned_uk_data.csv
   - rules_apriori_filtered.csv


C:\Users\acer\AppData\Local\Temp\ipykernel_9700\3178492681.py:17: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



⚙️ Đang xử lý dữ liệu & Feature Engineering...
   -> Số khách hàng hợp lệ sau khi lọc: 3920
   -> Đang mapping đặc trưng từ luật kết hợp...

⚔️ Đang so sánh K-Means, Agglomerative và DBSCAN (K=4)...


🧊 Vẽ biểu đồ 3D tương tác...


🕸️ Vẽ Radar Chart...


🐍 Vẽ Snake Plot...



📋 BÁO CÁO HỒ SƠ KHÁCH HÀNG (PROFILING):


,Recency,Frequency,Monetary,Số lượng KH
Cluster,,,,
0,182.90,1.30,331.30,1460
1,11.60,13.70,7255.30,632
2,19.30,2.10,526.50,776
3,68.60,4.20,1740.20,1052



✅ HOÀN TẤT BẢN FINAL! Đã xuất toàn bộ biểu đồ tương tác.
